In [94]:
import spacy
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import traceback

# Using CODING and ALGORITHMS to detect names
nlp = spacy.load('en_core_web_sm')

In [34]:
def filter_names(soup):
    for line in soup.text.split("\n"):
        if not line: continue

        doc = nlp(line)
        pronoun_counter = 0
        for token in doc:
            if token.pos_ == "PROPN": pronoun_counter += 1

        if float(pronoun_counter/len(doc)) == 1 and len(doc) > 1:
            print("*"*20)
            print(line)

    #     parse = HumanName(line)

In [2]:
from urllib.parse import urlparse

url = urlparse('asdf tha tad ')
url

ParseResult(scheme='', netloc='', path='asdf tha tad ', params='', query='', fragment='')

In [152]:
import re
def base_url(url):
    """Get the "homepage" given any url
    """
    url = urlparse(url)
    return(url.scheme+ "://" + url.netloc)

def parse_name(name):
    filter_name = re.compile('[;,.]*$') # get rid of trailing comma and period
    name = filter_name.sub("", name.strip())
    
    name = nlp(name)
    for tok in name:
#         print(tok, tok.pos_)
        if tok.pos_ not in ['PROPN', 'PUNCT', 'X']:
            raise ValueError("Not letting through!")
    return(name)

def html_only(soup):
    """
    Takes in soup and returns soup with script style and comments removed
    """
    blacklisted = ["script", "style", "comment"]
    for tag in soup.findAll():
            if tag.name.lower() in blacklisted:
                tag.extract()
    return(soup)

def get_page(url):
    page = requests.get(url)
    print("fetched", url)
    soup = BeautifulSoup(page.content, 'html.parser')
    return(html_only(soup))

def get_faculty_individual_pages(soup, url):
    global faculty
    global faculties
    
    baseurl = base_url(url)
    
    regex = re.compile('.+name$')   # fullname, prof-name, username, member-name, etc
    faculties = soup.find_all(class_=regex)
    faculty_list = []
    entry = {}
    
    for faculty in faculties:
        name = faculty.text
        try: name = parse_name(name)
        except: continue # false alarm for a name. check parse_name()
        entry['name']  = str(name)

        try:
            if faculty.name == 'a': pass
            elif faculty.find('a'): # if <a> tag is not a child, it better be a parent!
                faculty = faculty.find("a")
            else:
                while faculty.find('a'): # .name here is name of the tag!
                    faculty = faculty.parent
                faculty = faculty.find('a')
            # Now that we have the tag located
            url = faculty['href']
            if url[0] == '/': # sometimes href is relative to the homepage
                url = baseurl + url
                entry['url'] =  url
            else:
                entry['url'] = url
            if 'http' not in url: 
                continue # skip this "professor" :D

        except Exception as e:
#             print(e)
#             traceback.print_tb(e.__traceback__)
            continue

        faculty_list.append(dict(entry))
    
    return(faculty_list)

In [154]:
pages = pd.read_csv("people_pages.csv", header=None)

faculty_data = []
for url in pages[0][:]:
    soup = get_page(url)
    faculty_data += get_faculty_individual_pages(soup, url)

faculty_data

fetched https://www.ntnu.edu/physics/employees
fetched https://physics.yale.edu/people
fetched https://www9.physics.utoronto.ca/members/member_types/professor/
fetched https://www.phas.ubc.ca/researchers
fetched https://www.physics.yorku.ca/people-finder-who-does-what/


[{'name': 'Yoram Alhassid',
  'url': 'https://physics.yale.edu/people/yoram-alhassid'},
 {'name': 'Thomas Appelquist',
  'url': 'https://physics.yale.edu/people/thomas-appelquist'},
 {'name': 'Keith Baker', 'url': 'https://physics.yale.edu/people/keith-baker'},
 {'name': 'Charles Baltay',
  'url': 'https://physics.yale.edu/people/charles-baltay'},
 {'name': 'Sean Barrett',
  'url': 'https://physics.yale.edu/people/sean-barrett'},
 {'name': 'Sidney Cahn', 'url': 'https://physics.yale.edu/people/sidney-cahn'},
 {'name': 'Helen Caines',
  'url': 'https://physics.yale.edu/people/helen-caines'},
 {'name': 'Meng Cheng', 'url': 'https://physics.yale.edu/people/meng-cheng'},
 {'name': 'Eduardo da Silva Neto',
  'url': 'https://physics.yale.edu/people/eduardo-da-silva-neto'},
 {'name': 'Sarah Demers',
  'url': 'https://physics.yale.edu/people/sarah-demers'},
 {'name': 'David DeMille',
  'url': 'https://physics.yale.edu/people/david-demille'},
 {'name': 'Bonnie Fleming',
  'url': 'https://physic

In [172]:
faculty_data.to_csv("faculty_personal_website.csv")

fetched https://www.physics.yorku.ca/people-finder-who-does-what/
